In [128]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

from pandas.testing import assert_frame_equal
from lightgbm import LGBMRegressor

import math

In [129]:
sales = pd.read_csv(r"C:\Users\pc\Desktop\Data Science Folder\Retail1\sales data-set.csv")

stores = pd.read_csv(r"C:\Users\pc\Desktop\Data Science Folder\Retail1\stores data-set.csv")

features = pd.read_csv(r"C:\Users\pc\Desktop\Data Science Folder\Retail1\Features data set.csv")

In [130]:
# discovering the data

In [131]:

print('\n******* sales ********\n', 'shape: ', sales.shape, '\n', sales.columns, '\n', sales.describe(), '\n', sales.isnull().sum() )
print( '\n****** stores *******\n', 'shape: ', stores.shape, '\n',stores.columns, '\n',  stores.describe(), '\n', stores.isnull().sum() )
print('\n****** features *******\n', 'shape: ', features.shape, '\n',features.columns, '\n', features.describe(), '\n', features.isnull().sum() )



******* sales ********
 shape:  (421570, 5) 
 Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object') 
           Store      Dept  Weekly_Sales
count 421570.00 421570.00     421570.00
mean      22.20     44.26      15981.26
std       12.79     30.49      22711.18
min        1.00      1.00      -4988.94
25%       11.00     18.00       2079.65
50%       22.00     37.00       7612.03
75%       33.00     74.00      20205.85
max       45.00     99.00     693099.36 
 Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

****** stores *******
 shape:  (45, 3) 
 Index(['Store', 'Type', 'Size'], dtype='object') 
        Store      Size
count  45.00     45.00
mean   23.00 130287.60
std    13.13  63825.27
min     1.00  34875.00
25%    12.00  70713.00
50%    23.00 126512.00
75%    34.00 202307.00
max    45.00 219622.00 
 Store    0
Type     0
Size     0
dtype: int64

****** features *******
 shape:  (8190, 12) 
 Index(['Store

# only look after finding solutions to null values or negative ones after merging the data/filtering what does exist in test, because problems can go away that way  

# discovering the data that we're going to work with

In [132]:
train = sales.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
train = train.merge(stores, on='Store', how='left')

print( '***first 5 rows: ****\n: ', train.head() )
print( '****shape :  \n****', train.shape )
print( '*****describe:  \n****', train.describe() )
print( '****null values:   \n*****', train.isnull().sum() )


***first 5 rows: ****
:     Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1  05/02/2010      24924.50      False        42.31        2.57   
1      1     1  12/02/2010      46039.49       True        38.51        2.55   
2      1     1  19/02/2010      41595.55      False        39.93        2.51   
3      1     1  26/02/2010      19403.54      False        46.63        2.56   
4      1     1  05/03/2010      21827.90      False        46.50        2.62   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5    CPI  Unemployment  \
0        nan        nan        nan        nan        nan 211.10          8.11   
1        nan        nan        nan        nan        nan 211.24          8.11   
2        nan        nan        nan        nan        nan 211.29          8.11   
3        nan        nan        nan        nan        nan 211.32          8.11   
4        nan        nan        nan        nan        nan 211.35          8.11   

  Type 

In [133]:
train2 = train.copy()
MX= train2.copy()

MX['Weekly_Sales'][ MX['Weekly_Sales']< 0]=0

MX = MX.groupby(by=['Store', 'Dept', 'IsHoliday' ], as_index=False)['Weekly_Sales'].mean()



print(" ******* train describtion before:   \n" , train2.describe()['Weekly_Sales'].reset_index(), '\n')

for i in range(len(train)):
    if ( train2['Weekly_Sales'][i]<0 ):
        train2['Weekly_Sales'][i] = MX[ (MX['Store']==train2['Store'][i]) & (MX['Dept']==train2['Dept'][i]) & \
                                            (MX['IsHoliday']==train2['IsHoliday'][i]) ]['Weekly_Sales'] 
        

print(" ******* train describtion after:   \n" , train2.describe()['Weekly_Sales'].reset_index(), '\n' )


    
    


 ******* train describtion before:   
    index  Weekly_Sales
0  count     421570.00
1   mean      15981.26
2    std      22711.18
3    min      -4988.94
4    25%       2079.65
5    50%       7612.03
6    75%      20205.85
7    max     693099.36 

 ******* train describtion after:   
    index  Weekly_Sales
0  count     421570.00
1   mean      15985.94
2    std      22708.81
3    min          0.00
4    25%       2086.87
5    50%       7620.68
6    75%      20205.85
7    max     693099.36 



# how to set negative values in ONE column to zero

In [134]:
df = pd.DataFrame({'Col1': [10, -10, 15, 15, -15, -45],
                   'Col2': [13, 23, -18, 33, 14, 48],
                      'Col3': [17, 27, 22, 37, 28, 52]})
df

,Col1,Col2,Col3
0,10,13,17
1,-10,23,27
2,15,-18,22
3,15,33,37
4,-15,14,28
5,-45,48,52


In [135]:
df2 = df.copy()

df2['Col1'][df2['Col1']<0] = 0
df2



,Col1,Col2,Col3
0,10,13,17
1,0,23,27
2,15,-18,22
3,15,33,37
4,0,14,28
5,0,48,52


In [136]:
df2.describe()['Col1'].reset_index()

,index,Col1
0,count,6.00
1,mean,6.67
2,std,7.53
3,min,0.00
4,25%,0.00
5,50%,5.00
6,75%,13.75
7,max,15.00


# how to set negative values of one column to the mean of that column grouped by an other column (but after setting values to zero)

In [137]:
df = pd.DataFrame({'Col1': [10, 10, 15, 15, 15, -45],
                   'Col2': [13, 23, -18, 33, 14, 48],
                      'Col3': [-17, 27, -22, 37, 28, -52]})
df

,Col1,Col2,Col3
0,10,13,-17
1,10,23,27
2,15,-18,-22
3,15,33,37
4,15,14,28
5,-45,48,-52


In [138]:
df2 = df.copy()
df3 = df.copy()

print('\n****** this is df2 before ****** \n')
print ( df2 )

df3['Col3'][ df3['Col3']<0] = 0

test = df3.groupby('Col1', as_index=False)['Col3'].mean()

print('\n***** this is test ******* \n', test)

for i in range(len(df2['Col3'])):
    if ( df2['Col3'][i] < 0 ):
        df2['Col3'][i] = test[ test['Col1'] == df2['Col1'][i]]['Col3']

print('\n***** this is df2 after ******* \n')
print ( df2 )






****** this is df2 before ****** 

   Col1  Col2  Col3
0    10    13   -17
1    10    23    27
2    15   -18   -22
3    15    33    37
4    15    14    28
5   -45    48   -52

***** this is test ******* 
    Col1  Col3
0   -45  0.00
1    10 13.50
2    15 21.67

***** this is df2 after ******* 

   Col1  Col2  Col3
0    10    13    13
1    10    23    27
2    15   -18    21
3    15    33    37
4    15    14    28
5   -45    48     0


In [139]:
MX= train2.copy()

MX['MarkDown2'][ MX['MarkDown2']< 0]=0
MX['MarkDown3'][ MX['MarkDown3']<0]=0

M2 = MX.groupby(by=['Store', 'Dept', 'IsHoliday' ], as_index=False)['MarkDown2'].mean()
M3 = MX.groupby(by=['Store', 'Dept', 'IsHoliday' ], as_index=False)['MarkDown3'].mean()



print(" ******* train describtion before:   \n" , train2.describe()['MarkDown2'].reset_index(), '\n', train2.describe()['MarkDown3'].reset_index() )

for i in range(len(train)):
    if ( train2['MarkDown2'][i]<0 ):
        train2['MarkDown2'][i] = M2[ (M2['Store']==train2['Store'][i]) & (M2['Dept']==train2['Dept'][i]) & \
                                            (M2['IsHoliday']==train2['IsHoliday'][i]) ]['MarkDown2'] 
for i in range(len(train)):
    if ( train2['MarkDown3'][i]<0 ):
        train2['MarkDown3'][i] = M3[ (M3['Store']==train2['Store'][i]) & (M3['Dept']==train2['Dept'][i]) & \
                                            (M3['IsHoliday']==train2['IsHoliday'][i]) ]['MarkDown3'] 

print(" ******* train describtion after:   \n" , train2.describe()['MarkDown2'].reset_index(), '\n', train2.describe()['MarkDown3'].reset_index() )


 ******* train describtion before:   
    index  MarkDown2
0  count  111248.00
1   mean    3334.63
2    std    9475.36
3    min    -265.76
4    25%      41.60
5    50%     192.00
6    75%    1926.94
7    max  104519.54 
    index  MarkDown3
0  count  137091.00
1   mean    1439.42
2    std    9623.08
3    min     -29.10
4    25%       5.08
5    50%      24.60
6    75%     103.99
7    max  141630.61
 ******* train describtion after:   
    index  MarkDown2
0  count  111248.00
1   mean    3385.79
2    std    9490.46
3    min       0.00
4    25%      45.00
5    50%     207.40
6    75%    2144.70
7    max  104519.54 
    index  MarkDown3
0  count  137091.00
1   mean    1439.66
2    std    9623.04
3    min       0.00
4    25%       5.16
5    50%      24.82
6    75%     104.43
7    max  141630.61


In [140]:
train2.isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday            0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
dtype: int64

In [141]:
train2['MarkDown1'] = train2['MarkDown1'].fillna(0)
train2['MarkDown2'] = train2['MarkDown2'].fillna(0)
train2['MarkDown3'] = train2['MarkDown3'].fillna(0)
train2['MarkDown4'] = train2['MarkDown4'].fillna(0)
train2['MarkDown5'] = train2['MarkDown5'].fillna(0)

In [142]:
train2.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
dtype: int64

In [143]:
train2.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size'],
      dtype='object')

In [144]:
train2['Type'] = LabelEncoder().fit_transform(train2['Type'])
train2.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,05/02/2010,24924.50,False,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,0,151315
1,1,1,12/02/2010,46039.49,True,38.51,2.55,0.00,0.00,0.00,0.00,0.00,211.24,8.11,0,151315
2,1,1,19/02/2010,41595.55,False,39.93,2.51,0.00,0.00,0.00,0.00,0.00,211.29,8.11,0,151315
3,1,1,26/02/2010,19403.54,False,46.63,2.56,0.00,0.00,0.00,0.00,0.00,211.32,8.11,0,151315
4,1,1,05/03/2010,21827.90,False,46.50,2.62,0.00,0.00,0.00,0.00,0.00,211.35,8.11,0,151315


In [145]:
#train2['IsHolidayInt']  = [ int(x) for x in train2['IsHoliday']] 
train2 = train2.drop(['IsHoliday'], axis=1)
train2.head()

,Store,Dept,Date,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,05/02/2010,24924.50,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,0,151315
1,1,1,12/02/2010,46039.49,38.51,2.55,0.00,0.00,0.00,0.00,0.00,211.24,8.11,0,151315
2,1,1,19/02/2010,41595.55,39.93,2.51,0.00,0.00,0.00,0.00,0.00,211.29,8.11,0,151315
3,1,1,26/02/2010,19403.54,46.63,2.56,0.00,0.00,0.00,0.00,0.00,211.32,8.11,0,151315
4,1,1,05/03/2010,21827.90,46.50,2.62,0.00,0.00,0.00,0.00,0.00,211.35,8.11,0,151315


In [146]:
train3 = train2.copy()
train3['Date'] = pd.to_datetime(train3.Date,format="%d/%m/%Y")

train3['month'] = train3['Date'].dt.month
train3['year'] = train3['Date'].dt.year
train3['week'] = train3['Date'].dt.week




In [147]:
train3 = train3.drop(['Date'], axis=1)


In [148]:
train3.head()

,Store,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,month,year,week
0,1,1,24924.50,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,0,151315,2,2010,5
1,1,1,46039.49,38.51,2.55,0.00,0.00,0.00,0.00,0.00,211.24,8.11,0,151315,2,2010,6
2,1,1,41595.55,39.93,2.51,0.00,0.00,0.00,0.00,0.00,211.29,8.11,0,151315,2,2010,7
3,1,1,19403.54,46.63,2.56,0.00,0.00,0.00,0.00,0.00,211.32,8.11,0,151315,2,2010,8
4,1,1,21827.90,46.50,2.62,0.00,0.00,0.00,0.00,0.00,211.35,8.11,0,151315,3,2010,9


In [149]:
# NOTE: in this case: we need sales per week, so we won't do ['sum', 'mean'] columns

train3 = train3[['year', 'month', 'week', 'Store', 'Dept', 'Type', 'Size', 'Temperature', 'Fuel_Price',  'CPI', 'Unemployment','MarkDown1', 'MarkDown2', 'MarkDown3'\
                , 'MarkDown4', 'MarkDown5', 'Weekly_Sales']]

train3 = train3.sort_values(['year','month', 'week'])
train3

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Weekly_Sales
0,2010,2,5,1,1,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,24924.50
143,2010,2,5,1,2,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,50605.27
286,2010,2,5,1,3,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,13740.12
429,2010,2,5,1,4,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,39954.04
572,2010,2,5,1,5,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,32229.38
715,2010,2,5,1,6,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,5749.03
858,2010,2,5,1,7,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,21084.08
1001,2010,2,5,1,8,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,40129.01
1144,2010,2,5,1,9,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,16930.99
1287,2010,2,5,1,10,0,151315,42.31,2.57,211.10,8.11,0.00,0.00,0.00,0.00,0.00,30721.50


In [150]:

Store_ids = train3['Store'].unique()
Dept_ids = train3['Dept'].unique()
empty_df = []

for i in range (2010,2012):
    for j in range(0,13):
        for k in range(1, 52):
            for store in Store_ids:
                for dept in Dept_ids:
                    empty_df.append([i, j, k, store, dept])
#2012
for j in range(0,11):
    for k in range(1, 52):
        for store in Store_ids:
            for dept in Dept_ids:
                empty_df.append([2012, j, k, store, dept])

    
empty_df = pd.DataFrame(empty_df, columns=['year', 'month', 'week', 'Store','Dept'])
empty_df



,year,month,week,Store,Dept
0,2010,0,1,1,1
1,2010,0,1,1,2
2,2010,0,1,1,3
3,2010,0,1,1,4
4,2010,0,1,1,5
5,2010,0,1,1,6
6,2010,0,1,1,7
7,2010,0,1,1,8
8,2010,0,1,1,9
9,2010,0,1,1,10


In [151]:
train4 = train3.copy()

train4 = pd.merge(empty_df, train4, on=['year', 'month', 'week', 'Store','Dept'], how='left')
train4.fillna(0, inplace=True)



In [152]:
print( train4.head() )

print( 'train4 shape:  ', train4.shape ) # became 6 million rows

   year  month  week  Store  Dept  Type  Size  Temperature  Fuel_Price  CPI  \
0  2010      0     1      1     1  0.00  0.00         0.00        0.00 0.00   
1  2010      0     1      1     2  0.00  0.00         0.00        0.00 0.00   
2  2010      0     1      1     3  0.00  0.00         0.00        0.00 0.00   
3  2010      0     1      1     4  0.00  0.00         0.00        0.00 0.00   
4  2010      0     1      1     5  0.00  0.00         0.00        0.00 0.00   

   Unemployment  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5  \
0          0.00       0.00       0.00       0.00       0.00       0.00   
1          0.00       0.00       0.00       0.00       0.00       0.00   
2          0.00       0.00       0.00       0.00       0.00       0.00   
3          0.00       0.00       0.00       0.00       0.00       0.00   
4          0.00       0.00       0.00       0.00       0.00       0.00   

   Weekly_Sales  
0          0.00  
1          0.00  
2          0.00  
3       

# eliminate outliers

In [153]:
train4 = train4[(train4['Weekly_Sales'] <= 100000) & (train4['MarkDown1']<=10000) & (train4['MarkDown2']<=10000) \
               & (train4['MarkDown3']<=10000) & (train4['MarkDown4']<=10000) & (train4['MarkDown5']<=10000)]


In [154]:
train4['Weekly_Sales_month'] =  train4.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(-1)


In [155]:
train4.head()

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Weekly_Sales,Weekly_Sales_month
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [156]:
# always year before month in sort_values
gp_Weekly_Sales = train4.groupby(['Store', 'Dept'], as_index=False).agg({'Weekly_Sales':[np.min, np.max, np.mean, np.std]})

gp_Weekly_Sales.head()


Store Dept Weekly_Sales                          
                     amin     amax    mean      std
0     1    1         0.00 57258.43 1445.43  5978.74
1     1    2         0.00 65615.36 2996.18 11368.47
2     1    3         0.00 51159.17  837.68  3826.76
3     1    4         0.00 47893.23 2387.70  9062.17
4     1    5         0.00 85676.09 1524.47  6362.50

In [157]:
gp_Weekly_Sales.columns = ['Store', 'Dept', 'min_Weekly_Sales', 'max_Weekly_Sales', 'mean_Weekly_Sales', 'std_Weekly_Sales' ]

train4 = pd.merge(train4, gp_Weekly_Sales, on=['Store', 'Dept'] , how='left')
train4

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Weekly_Sales,Weekly_Sales_month,min_Weekly_Sales,max_Weekly_Sales,mean_Weekly_Sales,std_Weekly_Sales
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,57258.43,1445.43,5978.74
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,65615.36,2996.18,11368.47
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,51159.17,837.68,3826.76
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,47893.23,2387.70,9062.17
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,85676.09,1524.47,6362.50
5,2010,0,1,1,6,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28497.52,311.30,1445.04
6,2010,0,1,1,7,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,96198.12,1449.02,6290.92
7,2010,0,1,1,8,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42663.75,2312.50,8770.29
8,2010,0,1,1,9,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50675.72,1815.69,7194.64
9,2010,0,1,1,10,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42875.11,1997.09,7597.75


In [158]:
lag_list = [1, 2, 3]

for lag in lag_list:
    ft_name = ('Weekly_Sales_shifted%s' % lag)
    train4[ft_name] = train4.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(lag)
    # Fill the empty shifted features with 0
    train4[ft_name].fillna(0, inplace=True)

train4.head()

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,MarkDown5,Weekly_Sales,Weekly_Sales_month,min_Weekly_Sales,max_Weekly_Sales,mean_Weekly_Sales,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,57258.43,1445.43,5978.74,0.00,0.00,0.00
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,65615.36,2996.18,11368.47,0.00,0.00,0.00
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,51159.17,837.68,3826.76,0.00,0.00,0.00
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,47893.23,2387.70,9062.17,0.00,0.00,0.00
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,85676.09,1524.47,6362.50,0.00,0.00,0.00


In [159]:
train4['Weekly_Sales_trend'] = train4['Weekly_Sales']

for lag in lag_list:
    ft_name = ('Weekly_Sales_shifted%s' % lag)
    train4['Weekly_Sales_trend'] -= train4[ft_name]

train4['Weekly_Sales_trend'] /= len(lag_list) + 1

train4.head()

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,Weekly_Sales,Weekly_Sales_month,min_Weekly_Sales,max_Weekly_Sales,mean_Weekly_Sales,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3,Weekly_Sales_trend
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,57258.43,1445.43,5978.74,0.00,0.00,0.00,0.00
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,65615.36,2996.18,11368.47,0.00,0.00,0.00,0.00
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,51159.17,837.68,3826.76,0.00,0.00,0.00,0.00
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,47893.23,2387.70,9062.17,0.00,0.00,0.00,0.00
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,85676.09,1524.47,6362.50,0.00,0.00,0.00,0.00


In [160]:
train4 = train4.fillna(0)

train4.head()

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,Weekly_Sales,Weekly_Sales_month,min_Weekly_Sales,max_Weekly_Sales,mean_Weekly_Sales,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3,Weekly_Sales_trend
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,57258.43,1445.43,5978.74,0.00,0.00,0.00,0.00
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,65615.36,2996.18,11368.47,0.00,0.00,0.00,0.00
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,51159.17,837.68,3826.76,0.00,0.00,0.00,0.00
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,47893.23,2387.70,9062.17,0.00,0.00,0.00,0.00
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,85676.09,1524.47,6362.50,0.00,0.00,0.00,0.00


In [161]:
# Shop mean encoding
gp_store_mean = train4.groupby(['Store']).agg({'Weekly_Sales': ['mean']})
gp_store_mean.columns = ['store_mean']
gp_store_mean.reset_index(inplace=True)
print( gp_store_mean )

train4 = pd.merge(train4, gp_store_mean, on=['Store'], how='left')


    Store  store_mean
0       1     1023.26
1       2     1188.16
2       3      297.68
3       4     1281.87
4       5      285.71
5       6     1192.30
6       7      473.71
7       8      721.57
8       9      485.78
9      10     1200.88
10     11     1063.03
11     12      726.54
12     13     1171.40
13     14     1113.60
14     15      540.11
15     16      465.21
16     17      751.17
17     18      854.61
18     19      980.02
19     20     1232.30
20     21      617.39
21     22      783.95
22     23     1144.02
23     24      902.44
24     25      575.44
25     26      800.63
26     27     1081.04
27     28      971.06
28     29      474.41
29     30      405.26
30     31      888.88
31     32      911.98
32     33      238.57
33     34      826.15
34     35      708.16
35     36      345.23
36     37      479.51
37     38      356.00
38     39      887.01
39     40      782.25
40     41      902.01
41     42      505.23
42     43      583.70
43     44      279.95
44     45 

In [162]:
# Dept mean encoding
gp_dept_mean = train4.groupby(['Dept']).agg({'Weekly_Sales': ['mean']})
gp_dept_mean.columns = ['dept_mean']
gp_dept_mean.reset_index(inplace=True)
print( gp_dept_mean )

train4 = pd.merge(train4, gp_dept_mean, on=['Dept'], how='left')

    Dept  dept_mean
0      1    1222.04
1      2    2631.72
2      3     748.86
3      4    1681.12
4      5    1255.58
5      6     290.64
6      7    1311.74
7      8    1956.34
8      9    1303.56
9     10    1182.79
10    11     938.41
11    12     274.65
12    13    1989.61
13    14     941.45
14    16     950.10
15    17     653.95
16    18     347.46
17    19      69.01
18    20     333.33
19    21     325.79
20    22     590.47
21    23    1365.05
22    24     342.39
23    25     575.26
24    26     440.29
25    27      87.79
26    28      38.09
27    29     296.87
28    30     221.10
29    31     143.42
..   ...        ...
51    54       5.94
52    55     588.62
53    56     237.05
54    58     159.98
55    59      42.45
56    60      20.56
57    65      71.01
58    67     480.02
59    71     298.18
60    72    2364.79
61    74     881.71
62    77       0.14
63    78       0.02
64    79    1417.49
65    80     729.37
66    81    1011.92
67    82     976.80
68    83     200.84


In [163]:
# store with dept mean encoding.
gp_store_dept_mean = train4.groupby(['Store', 'Dept']).agg({'Weekly_Sales': ['mean']})
gp_store_dept_mean.columns = ['store_dept_mean']
gp_store_dept_mean.reset_index(inplace=True)
train4 = pd.merge(train4, gp_store_dept_mean, on=['Store', 'Dept'], how='left')
train4.head()



,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,max_Weekly_Sales,mean_Weekly_Sales,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3,Weekly_Sales_trend,store_mean,dept_mean,store_dept_mean
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,57258.43,1445.43,5978.74,0.00,0.00,0.00,0.00,1023.26,1222.04,1445.43
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,65615.36,2996.18,11368.47,0.00,0.00,0.00,0.00,1023.26,2631.72,2996.18
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,51159.17,837.68,3826.76,0.00,0.00,0.00,0.00,1023.26,748.86,837.68
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,47893.23,2387.70,9062.17,0.00,0.00,0.00,0.00,1023.26,1681.12,2387.70
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,85676.09,1524.47,6362.50,0.00,0.00,0.00,0.00,1023.26,1255.58,1524.47


In [164]:
# Year mean encoding.
gp_year_mean = train4.groupby(['year']).agg({'Weekly_Sales': ['mean']})
gp_year_mean.columns = ['year_mean']
gp_year_mean.reset_index(inplace=True)
train4 = pd.merge(train4, gp_year_mean, on=['year'], how='left')

train4.head()

,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,mean_Weekly_Sales,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3,Weekly_Sales_trend,store_mean,dept_mean,store_dept_mean,year_mean
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,1445.43,5978.74,0.00,0.00,0.00,0.00,1023.26,1222.04,1445.43,832.72
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,2996.18,11368.47,0.00,0.00,0.00,0.00,1023.26,2631.72,2996.18,832.72
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,837.68,3826.76,0.00,0.00,0.00,0.00,1023.26,748.86,837.68,832.72
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,2387.70,9062.17,0.00,0.00,0.00,0.00,1023.26,1681.12,2387.70,832.72
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,1524.47,6362.50,0.00,0.00,0.00,0.00,1023.26,1255.58,1524.47,832.72


In [165]:
# Month mean encoding.
gp_month_mean = train4.groupby(['month']).agg({'Weekly_Sales': ['mean']})
gp_month_mean.columns = ['month_mean']
gp_month_mean.reset_index(inplace=True)
train4 = pd.merge(train4, gp_month_mean, on=['month'], how='left')

train4.head()


,year,month,week,Store,Dept,Type,Size,Temperature,Fuel_Price,CPI,...,std_Weekly_Sales,Weekly_Sales_shifted1,Weekly_Sales_shifted2,Weekly_Sales_shifted3,Weekly_Sales_trend,store_mean,dept_mean,store_dept_mean,year_mean,month_mean
0,2010,0,1,1,1,0.00,0.00,0.00,0.00,0.00,...,5978.74,0.00,0.00,0.00,0.00,1023.26,1222.04,1445.43,832.72,0.00
1,2010,0,1,1,2,0.00,0.00,0.00,0.00,0.00,...,11368.47,0.00,0.00,0.00,0.00,1023.26,2631.72,2996.18,832.72,0.00
2,2010,0,1,1,3,0.00,0.00,0.00,0.00,0.00,...,3826.76,0.00,0.00,0.00,0.00,1023.26,748.86,837.68,832.72,0.00
3,2010,0,1,1,4,0.00,0.00,0.00,0.00,0.00,...,9062.17,0.00,0.00,0.00,0.00,1023.26,1681.12,2387.70,832.72,0.00
4,2010,0,1,1,5,0.00,0.00,0.00,0.00,0.00,...,6362.50,0.00,0.00,0.00,0.00,1023.26,1255.58,1524.47,832.72,0.00


In [166]:
print('\n******* train4 ********\n', 'shape: ', train4.shape, '\n', train4.columns, '\n', train4.describe(), '\n', train4.isnull().sum() )



******* train4 ********
 shape:  (6826487, 31) 
 Index(['year', 'month', 'week', 'Store', 'Dept', 'Type', 'Size', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales',
       'Weekly_Sales_month', 'min_Weekly_Sales', 'max_Weekly_Sales',
       'mean_Weekly_Sales', 'std_Weekly_Sales', 'Weekly_Sales_shifted1',
       'Weekly_Sales_shifted2', 'Weekly_Sales_shifted3', 'Weekly_Sales_trend',
       'store_mean', 'dept_mean', 'store_dept_mean', 'year_mean',
       'month_mean'],
      dtype='object') 
             year      month       week      Store       Dept       Type  \
count 6826487.00 6826487.00 6826487.00 6826487.00 6826487.00 6826487.00   
mean     2010.94       5.70      26.01      23.02      46.05       0.03   
std         0.80       3.61      14.71      12.99      29.19       0.21   
min      2010.00       0.00       1.00       1.00       1.00       0.00   
25%      2010.00       3.00   

In [167]:
del(gp_store_mean, gp_store_dept_mean, gp_year_mean, gp_month_mean )


In [168]:

"""
# for pandas dataframes we use & for and | for or
train_set = train14[ np.logical_or( (train14['month']>= 3 & train14['year']==2013), (train14['year'] == 2014),(train14['month']<= 3 & train14['year']==2015)) ]
train_set.shape
"""

L1 = train4[ (train4['year']==2010) & (train4['month']>=4) ]
L2 = train4[ train4['year']==2011 ]
L3 = train4[ (train4['year']==2012) & (train4['month']<=4) ]

L1 = L1.append(L2)
L1 = L1.append(L3)

train_set = L1.copy()
print( train_set.shape )

del(L1,L2,L3)

validation_set = train4[(train4['year']==2012) & (train4['month'] >=5)]
print ( validation_set.shape)



(4988774, 31)
(1094436, 31)


In [169]:
X_train = train_set.drop(['Weekly_Sales'], axis=1)
Y_train = train_set['Weekly_Sales'].astype(int)
#X_validation = validation_set.drop(['Weekly_Sales'], axis=1)
#Y_validation = validation_set['Weekly_Sales'].astype(int)


In [170]:
int_features = ['Store', 'Dept', 'year', 'month']

#X_train[int_features] = X_train[int_features].astype('int32')
#X_validation[int_features] = X_validation[int_features].astype('int32')

# random forest regressor

In [171]:
rf_features = X_train.columns


rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
#rf_test = X_test[rf_features]
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(rf_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [172]:

rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)
#rf_test_pred = rf_model.predict(rf_test)

rmse = np.sqrt(mean_squared_error(rf_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(rf_val_pred, Y_validation))
print('rmse of validation : ' , rmse)



rmse of train :  681.9565280508592
rmse of validation :  4068.609240939771


In [173]:
print(' score of train:    ', rf_model.score(rf_train, Y_train))
print(' score of validation:  ', rf_model.score(rf_val, Y_validation) )


 score of train:     0.9844294395916822
 score of validation:   0.9461894129284077


# linear regression

In [174]:
#lr_features = ['item_cnt', 'item_cnt_shifted1', 'item_trend', 'mean_item_cnt', 'shop_mean']

lr_features = X_train.columns


lr_train = X_train[lr_features]
lr_val = X_validation[lr_features]
#lr_test = X_test[lr_features]

In [175]:
lr_scaler = MinMaxScaler()
lr_scaler.fit(lr_train)
lr_train = lr_scaler.transform(lr_train)
lr_val = lr_scaler.transform(lr_val)
#lr_test = lr_scaler.transform(lr_test)
lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(lr_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [176]:
lr_train_pred = lr_model.predict(lr_train)
lr_val_pred = lr_model.predict(lr_val)
#lr_test_pred = lr_model.predict(lr_test)

In [177]:
rmse = np.sqrt(mean_squared_error(lr_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(lr_val_pred, Y_validation))
print('rmse of validation : ' , rmse)

rmse of train :  0.0726849860877738
rmse of validation :  0.3291022596939236


In [178]:
print(' score of train:    ', lr_model.score(lr_train, Y_train))
print(' score of validation:  ', lr_model.score(lr_val, Y_validation) )

 score of train:     0.9999999998231194
 score of validation:   0.9999999996479231


# knn

In [179]:
# Use only part of features on KNN.
#knn_features = ['item_cnt', 'item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1',
 #               'item_cnt_shifted2', 'shop_mean', 'shop_item_mean', 
  #              'item_trend', 'mean_item_cnt']


knn_features = X_train.columns



# Subsample train set (using the whole data was taking too long).
#X_train_sampled = X_train[:100000]
#Y_train_sampled = Y_train[:100000]

knn_train = X_train[knn_features]
knn_val = X_validation[knn_features]
#knn_test = X_test[knn_features]

In [ ]:
knn_scaler = MinMaxScaler()
knn_scaler.fit(knn_train)
knn_train = knn_scaler.transform(knn_train)
knn_val = knn_scaler.transform(knn_val)
#knn_test = knn_scaler.transform(knn_test)

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=9, leaf_size=13, n_jobs=-1)
knn_model.fit(knn_train, Y_train)

In [ ]:
knn_train_pred = knn_model.predict(knn_train)
knn_val_pred = knn_model.predict(knn_val)
#knn_test_pred = knn_model.predict(knn_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(knn_train_pred, Y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(knn_val_pred, Y_validation))
print('rmse of validation : ' , rmse)

print(' score of train:    ', knn_model.score(knn_train, Y_train))
print(' score of validation:  ', knn_model.score(knn_val, Y_validation) )

# lightgbm

In [ ]:
lgbm_model=LGBMRegressor(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=32,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.04,
        reg_lambda=0.073,
        min_split_gain=0.0222415,
        min_child_weight=40)

In [ ]:
lgbm_model.fit(X_train, Y_train)


lightgbm_train_pred = lgbm_model.predict(X_train)
lightgbm_val_pred = lgbm_model.predict(X_validation)
#lightgbm_test_pred = lgbm_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(Y_train, lightgbm_train_pred ))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(Y_validation, lightgbm_val_pred))
print('rmse of validation : ' , rmse)

In [ ]:
print( lgbm_model.score(X_train, lightgbm_train_pred) )
print( lgbm_model.score(X_validation, lightgbm_val_pred) )

In [ ]:
print( Y_train.shape, '    ',  lightgbm_train_pred.shape )
print(Y_validation.shape ,'      ',  lightgbm_val_pred.shape )
print( X_train.shape, lightgbm_train_pred.shape )